In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CAPSTONE WORK/data_male_female.csv')

In [ ]:
!pip install gender_guesser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.3/379.3 kB 15.7 MB/s eta 0:00:00


In [ ]:
import gender_guesser.detector as gender
d = gender.Detector(case_sensitive=False)
fullname = []
art_id = []
date = []

for i in range(0, len(df)):
    full_names = str(df['author:string[]'].iloc[i]).split('|')
    for name in full_names:
        first_name = str(name).split(" ")[0]
        if d.get_gender(first_name) == 'male':
          fullname.append(name)
          art_id.append(df.iloc[i]["article:ID"])
          date.append(df.iloc[i]["mdate:date"])

In [ ]:
save = pd.DataFrame(zip(fullname, art_id, date), columns=["Name", "Article_ID", "Date"])

In [ ]:
save["Name"].duplicated(keep=False).value_counts()

True     2062201
False     327048
Name: Name, dtype: int64

In [ ]:
save["repeat"] = save["Name"].duplicated(keep=False)

In [ ]:
df = save[save.repeat == True]

In [ ]:
len(df)

2062201

In [ ]:
df.sort_values(by="Name")

,Name,Article_ID,Date,repeat
347041,Aad Correljé,7021149,2021-10-14,True
346723,Aad Correljé,7020740,2023-01-15,True
1647815,Aad J. van der Steen,8651944,2020-09-29,True
1871283,Aad J. van der Steen,8948302,2020-03-02,True
1530899,Aad J. van der Steen,8496680,2020-02-22,True
...,...,...,...,...
1893864,Ünal Çavusoglu,8979541,2022-06-13,True
2254546,Ünal Çavusoglu,9452691,2021-10-14,True
134485,Ünal Çavusoglu,6739371,2022-01-03,True
399133,Ünver Çinar,7084922,2020-02-21,True


In [ ]:
df.Date = pd.to_datetime(df.Date)

In [ ]:
min_dates = df.loc[df.groupby('Name').Date.idxmin()]
max_dates = df.loc[df.groupby('Name').Date.idxmax()]

In [ ]:
merged = min_dates.merge(max_dates, on='Name')
merged

,Name,Article_ID_x,Date_x,repeat_x,Article_ID_y,Date_y,repeat_y
0,Aad Correljé,7021149,2021-10-14,True,7020740,2023-01-15,True
1,Aad J. van der Steen,6810540,2019-10-24,True,8651944,2020-09-29,True
2,Aad Mathijssen,8074741,2020-10-26,True,7867975,2022-05-09,True
3,Aad P. A. van Moorsel,7070905,2017-05-17,True,8141519,2021-10-27,True
4,Aad Slootmaker,8289753,2020-09-29,True,8247766,2021-05-14,True
...,...,...,...,...,...,...,...
301499,Ünal Göktas,7338775,2018-08-13,True,9151667,2021-02-17,True
301500,Ünal Ufuktepe,7884794,2022-10-02,True,7884794,2022-10-02,True
301501,Ünal Çakiroglu,8644823,2020-02-18,True,9546777,2022-09-16,True
301502,Ünal Çavusoglu,7067764,2019-06-19,True,8979541,2022-06-13,True


In [ ]:
find_diff = merged[merged.Date_x != merged.Date_y]

In [ ]:
find_diff['date_diff'] = find_diff.Date_y - find_diff.Date_x

In [ ]:
find_diff

,Name,Article_ID_x,Date_x,repeat_x,Article_ID_y,Date_y,repeat_y,date_diff
0,Aad Correljé,7021149,2021-10-14,True,7020740,2023-01-15,True,458 days
1,Aad J. van der Steen,6810540,2019-10-24,True,8651944,2020-09-29,True,341 days
2,Aad Mathijssen,8074741,2020-10-26,True,7867975,2022-05-09,True,560 days
3,Aad P. A. van Moorsel,7070905,2017-05-17,True,8141519,2021-10-27,True,1624 days
4,Aad Slootmaker,8289753,2020-09-29,True,8247766,2021-05-14,True,227 days
...,...,...,...,...,...,...,...,...
301496,Ümit Önen,9552895,2021-10-14,True,9015231,2023-01-17,True,460 days
301497,Ümit Özgüner,9319395,2017-05-20,True,6616834,2022-11-13,True,2003 days
301499,Ünal Göktas,7338775,2018-08-13,True,9151667,2021-02-17,True,919 days
301501,Ünal Çakiroglu,8644823,2020-02-18,True,9546777,2022-09-16,True,941 days


In [ ]:
find_diff = find_diff.rename(columns={'date_diff': 'career_length'})

In [ ]:
find_diff.head(1)

,Name,Article_ID_x,Date_x,repeat_x,Article_ID_y,Date_y,repeat_y,career_length
0,Aad Correljé,7021149,2021-10-14,True,7020740,2023-01-15,True,458 days


In [ ]:
find_diff['Gender'] = 'M'

In [ ]:
find_diff.head(1)

,Name,Article_ID_x,Date_x,repeat_x,Article_ID_y,Date_y,repeat_y,career_length,Gender
0,Aad Correljé,7021149,2021-10-14,True,7020740,2023-01-15,True,458 days,M


In [ ]:
find_diff.to_csv('/content/drive/MyDrive/CAPSTONE WORK/topics data/career_length1_m.csv', encoding='utf-8', index=False)

In [ ]:
f = pd.read_csv('/content/drive/MyDrive/CAPSTONE WORK/topics data/career_length1.csv')

In [ ]:
f['Gender'] = 'F'

In [ ]:
f.head(1)

,Unnamed: 0,Name,Article_ID_x,Date_x,repeat_x,Article_ID_y,Date_y,repeat_y,career_length,Gender
0,0,Aarti Singh,6772987,2020-03-31,True,6631706,2020-04-16,True,16 days,F


In [ ]:
f = f.iloc[: , 1:]

In [ ]:
f.head(1)

,Name,Article_ID_x,Date_x,repeat_x,Article_ID_y,Date_y,repeat_y,career_length,Gender
0,Aarti Singh,6772987,2020-03-31,True,6631706,2020-04-16,True,16 days,F


In [ ]:
f.shape

(4539, 9)

In [ ]:
m = pd.read_csv('/content/drive/MyDrive/CAPSTONE WORK/topics data/career_length1_m.csv')

In [ ]:
m = m.sample(n=5461)

In [ ]:
m.shape

(5461, 9)

In [ ]:
mf_cl = f.append(m)

In [ ]:
mf_cl.shape

(10000, 9)

In [ ]:
mf_cl

,Name,Article_ID_x,Date_x,repeat_x,Article_ID_y,Date_y,repeat_y,career_length,Gender
0,Aarti Singh,6772987,2020-03-31,True,6631706,2020-04-16,True,16 days,F
1,Abbe Mowshowitz,6629835,2020-04-01,True,6796786,2020-09-24,True,176 days,F
2,Abha Aggarwal,6726836,2021-02-17,True,6642803,2021-09-16,True,211 days,F
3,Abigail Morrison,6627529,2020-05-28,True,6628283,2022-10-02,True,857 days,F
4,Abigail Sellen,6598090,2020-06-05,True,6697328,2022-11-13,True,891 days,F
...,...,...,...,...,...,...,...,...,...
60671,Dimitri Hendriks,7263483,2018-08-13,True,6577621,2022-10-02,True,1511 days,M
1186,Abdullah Asim Yilmaz,9029679,2020-10-26,True,9054829,2021-07-13,True,260 days,M
131040,Julien Gaulmin,7769096,2020-05-10,True,7286693,2021-03-24,True,318 days,M
68419,Emmanuel Tsekleves,8103651,2019-05-29,True,6904422,2022-10-02,True,1222 days,M


In [ ]:
result = df.groupby('Name')['Claimed'].mean()


In [ ]:
mf_cl.to_csv('/content/drive/MyDrive/CAPSTONE WORK/topics data/mf_cl.csv', encoding='utf-8', index=False)

In [52]:
mf_cl.shape

(10000, 9)

In [53]:
mf_cl['career_length'] = mf_cl['career_length'].str.replace(' days', '')

In [54]:
mf_cl['career_length']

0           16
1          176
2          211
3          857
4          891
          ... 
60671     1511
1186       260
131040     318
68419     1222
188672    2053
Name: career_length, Length: 10000, dtype: object

In [55]:
mf_cl['career_length'] = mf_cl['career_length'].astype(int)

In [58]:
result = mf_cl.groupby('Gender')['career_length'].mean().round(0)

In [59]:
result

Gender
F    589.0
M    807.0
Name: career_length, dtype: float64